In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from pkg_resources import find_eggs_in_zip
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import torch

In [ ]:
from kan import *

In [ ]:
device = torch.device('cuda')

In [ ]:
device

In [ ]:
ticker = "^GSPC"  # S&P 500
data = yf.download(ticker, start="2010-01-01", end="2023-01-01")

In [ ]:
data

In [ ]:
data['return'] = data['Adj Close'].pct_change()  #daily returns
data['sma_50'] = data['Adj Close'].rolling(window=50).mean()  # 50 day ma
data['sma_200'] = data['Adj Close'].rolling(window=200).mean()  # 200 day ma
data = data.dropna()

In [ ]:
X = data[['return', 'sma_50', 'sma_200']]
y = data['Adj Close'].shift(-1)  # next day's closing price

In [ ]:
X = X[:-1]
y = y[:-1]

In [ ]:
splitter = TimeSeriesSplit(n_splits=5)
model = LinearRegression()

In [ ]:
X

In [ ]:
y

In [ ]:
predictions = []
actuals = []
X_train, y_train, X_test, Y_test = [], [], [], []
for train_index, test_index in splitter.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
data.columns

In [ ]:
fig, axs = plt.subplots(5, 1, figsize=(15, 15), sharex=True)

fold = 0
data = data.sort_index()
for train_idx, val_idx in splitter.split(data):
    train, val = data.iloc[train_idx], data.iloc[val_idx]

    train['Close'].plot(ax=axs[fold], label='train_set', title=f'Data Split Fold {fold}')
    val['Close'].plot(ax=axs[fold], label='test_set')

    model.fit(train.drop['Close'], train['Close'] )
    y_pred=model.predict(val.drop['Close'])
    y_pred.plot(ax=axs[fold], label='predictions')

    axs[fold].axvline(val.index.min(), color='black', ls='--')
    fold += 1

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
predictions.extend(y_pred)
actuals.extend(y_test.values)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error for fold: {mse}')

In [ ]:
actuals

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(data.index[1:len(actuals) + 1], actuals, label='Actual')
plt.plot(data.index[1:len(predictions) + 1], predictions, label='Predicted', linestyle='dashed')
plt.xlabel('Date')
plt.ylabel('S&P 500 Price')
plt.title('S&P 500 Price Prediction')
plt.legend()
plt.show()

TWITTER

In [8]:
import tweepy
import os
from dotenv import load_dotenv
dotenv_path = '.env'
load_dotenv(dotenv_path)

True

In [6]:
auth = tweepy.OAuth1UserHandler(
    os.environ.get("consumer_key"), os.environ.get("consumer_secret"), os.environ.get("access_token"), os.environ.get("access_token_secret"))
api = tweepy.API(auth)

In [ ]:
results = api.search_tweets("S&P 500")

REDDIT

In [10]:
import praw
reddit = praw.Reddit(client_id=os.environ.get("client_id"), client_secret=os.environ.get("client_secret"), user_agent=os.environ.get("user_agent"))

In [ ]:
subreddit = reddit.subreddit("trading")
search_query = "forex"
for submission in subreddit.search(search_query, sort="relevance", limit=10):
    print(f"Title: {submission.title}")
    print(f"URL: {submission.url}")
    print(f"Score: {submission.score}")
    print(f"Created: {submission.created_utc}")
    print("-" * 40)

VADER

In [18]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [15]:
import pandas as pd
df = pd.DataFrame({"tweets": [
    "Forex trading tips: Always set stop-loss levels to manage risks.",
    "Looking to diversify your portfolio? Forex could be an option.",
    "Today's forex analysis: USD/EUR shows bullish trends. Stay tuned!",
    "Beginner's guide to forex: Start small, learn trends, and avoid greed.",
    "Forex insight: The impact of interest rate hikes on currency pairs.",
    "Why forex trading requires discipline and strategy.",
    "Top 5 forex pairs to watch this week: USD/EUR, GBP/USD, and more.",
    "Forex myths debunked: It's not a get-rich-quick scheme!",
    "Master forex trading by understanding economic indicators.",
    "Daily forex update: Volatility expected in GBP/USD due to market news."]
})

In [24]:
analyzer = SentimentIntensityAnalyzer()
for sentence in df['tweets']:
    vs = analyzer.polarity_scores(sentence)
    print("{:-<115} {}".format(sentence, str(vs)))

Forex trading tips: Always set stop-loss levels to manage risks.--------------------------------------------------- {'neg': 0.189, 'neu': 0.811, 'pos': 0.0, 'compound': -0.2732}
Looking to diversify your portfolio? Forex could be an option.----------------------------------------------------- {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Today's forex analysis: USD/EUR shows bullish trends. Stay tuned!-------------------------------------------------- {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Beginner's guide to forex: Start small, learn trends, and avoid greed.--------------------------------------------- {'neg': 0.353, 'neu': 0.647, 'pos': 0.0, 'compound': -0.5994}
Forex insight: The impact of interest rate hikes on currency pairs.------------------------------------------------ {'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'compound': 0.4588}
Why forex trading requires discipline and strategy.---------------------------------------------------------------- {'neg': 0.0

In [ ]:
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained("demo-org/finbert-pretrain", use_auth_token=True)